# Descarga de información del BOE sobre ofertas de empleo para servir mediante API Flask

Alumno: Diego Sánchez de la Fuente

Firma: Aseguro que todo el contenido expuesto en la práctica es propio y no está copiado de ninguna otra fuente aunque se haya podido utilizar documentación para la inspiración del mismo.

In [1]:
import requests, csv
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import urllib3 as url
import datetime
import string
from urllib.parse import urlparse, urlunparse
import regex, re
from flask import Flask, render_template, request, Response, jsonify #Utilidades para la construccion de un API Rest
import warnings
import os
from flask import Flask, render_template, request, send_from_directory
warnings.filterwarnings(action='ignore')

In [2]:
# Ejecutar antes del API Flask si se desea omitir la ejecución del modulo Scrapper
df = pd.read_csv('datos_offline/datos_obtenidos.boe.csv', sep='|')

In [3]:
class DescargaBOE:
    """
    Clase que permite la descarga del BOE en lo referente a las Resoluciones relacionadas con las convocatorias de Oposiciones
    Para instanciar la clase:
    MiClase = DescsargaBOE()
    Para fijar el Offset
    MiClase.establecer_offset(offset)
    """

    
    def __init__(self):
        """
        Generador de la clase no recibe parámetros
        establece las variables internas
        fecha_actual, url_patron, dominio u dataset con los boes
        """
        # Obtiene la fecha y hora actual
        self.fecha_actual = datetime.datetime.now()
        self.url_patron = string.Template("https://www.boe.es/boe/dias/$anio/$mes/$dia/index.php?s=2B")
        self.dominio = "https://www.boe.es"
        self.dataset_boes = pd.DataFrame({'url':[], 
                                          'titulo':[],
                                          'texto':[]})
        self.user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59"
        self.timeout = 100
        

    def quitar_etiquetas_html(self, cadena_html: str) -> str:
        """
        Método Helper para la eliminación de etiquetas HTML de los textos parseados
        uso:
        Entrada: Texto con etiquetas HTML
        Salida: Mismo Texto sin etiquetas HTML
        self.quitar_etiquetas_html(Texto)
        """
        # Parsear la cadena HTML
        soup = BeautifulSoup(cadena_html, 'html.parser')    
        # Obtener solo el texto sin etiquetas HTML
        texto = soup.get_text(separator='')
        texto = texto.replace('[', '')
        texto = texto.replace(']', '')
        return texto
    
    

    def establecer_offset(self, offset: int):
        """
        Método que estalece el OFFSET definido como el número de días a partir de la fecha
        actual desde la que se quiere descargar los BOES
        Si instanciamos
        MiClase.establecer_offset(5)
        Inspeccionaremos los BOES de hace 5 días
        Entrada: Offset Es un etero
        Salida: Variables internas de la clase (URLS de los BOES)
        """
        fecha_calculada = self.fecha_actual - datetime.timedelta(days=offset)      
        anio = fecha_calculada.year
        mes = str(fecha_calculada.month).zfill(2)
        dia = str(fecha_calculada.day).zfill(2)
        fecha = {'anio': anio,
                 'mes': mes,
                 'dia': dia}        
        self.url_busqueda = self.url_patron.substitute(anio=fecha['anio'],
                                                       mes=fecha['mes'],
                                                       dia=fecha['dia'])       



    def buscar_urls_xmls(self):
        """
        Con los parámetros obtenidos de establecer_offset, localizamos las URLS
        de las disposiciones relativas a las ofertas de empelo público es decir 
        Sección II B del BOE
        Uso
        self.buscar_urls_xmls()
        """
        
        url = self.url_busqueda
        parsed_url = urlparse(url)        
        
        dominio = parsed_url.netloc                
        
        try:
            response = requests.get(url)
            html_content = response.content        
            
            soup = BeautifulSoup(html_content, 'html.parser')       
            
            titulo_buscado = "Otros formatos"
                   
            enlaces_con_titulo = soup.find_all('a', string=titulo_buscado)
            
            lista_urls = []
            for enlace in enlaces_con_titulo:
                url_obtenida = f'https://{dominio}{enlace["href"]}'
            
                parsed_url = urlparse(url_obtenida)
                parsed_url_lista = list(parsed_url)
                parsed_url_lista[2] = 'diario_boe/xml.php'
            
                # Convertir la lista de nuevo a un objeto ParseResult
                parsed_url_modificada = urlparse(urlunparse(parsed_url_lista))
                lista_urls.append(urlunparse(parsed_url_modificada))
            
            self.lista_urls = lista_urls
        except:
            self.lista_urls =  []
        

    def obtener_lista_xmls(self):
        """
        Con los parámetros obtenidos de establecer_offset, localizamos los XMLs
        de las disposiciones relativas a las ofertas de empelo público es decir 
        Sección II B del BOE
        Uso
        self.obtener_lista_xmls()
        """
        lista_respuestas = []
        for url in self.lista_urls:
            #url = 'https://www.boe.es/diario_boe/xml.php?id=BOE-A-2021-10344'
            headers = {'accept': 'application/xml;q=0.9, */*;q=0.8',
                       'User-Agent': self.user_agent}
            try:                
                response = requests.get(url, headers=headers, 
                                        timeout=self.timeout                                    
                                       )
                lista_respuestas.append(response.text)
            except:
                print(f"Existe una URL {url} que no es posible descargar")
            
        self.lista_xmls = lista_respuestas
    
    
    def obtener_lista_titulos(self):
        """
        Con los parámetros obtenidos de establecer_offset, localizamos los titulos
        de las disposiciones relativas a las ofertas de empelo público es decir 
        Sección II B del BOE
        Uso
        self.obtener_lista_titulos()
        """
        lista_titulos = []
        for XML in self.lista_xmls:
            soup = BeautifulSoup(XML, "xml")
            titulo = soup.find("titulo")
            lista_titulos.append(titulo.get_text())
        self.lista_titulos = lista_titulos
        
    
    def obtener_lista_textos(self):
        """
        Con los parámetros obtenidos de establecer_offset, localizamos los textos
        de las disposiciones relativas a las ofertas de empelo público es decir 
        Sección II B del BOE
        Uso
        self.obtener_lista_textos()
        """
        lista_textos = []
        for XML in self.lista_xmls:
            textos = ""
            soup = BeautifulSoup(XML, "xml") 
            text = soup.find_all("texto")           
            lista_textos.append(str(text))
        self.lista_textos = lista_textos

    

    def obtener_lista_urls_pdf(self):
        """
        Con los parámetros obtenidos de establecer_offset, localizamos las urls pdfs
        de las disposiciones relativas a las ofertas de empelo público es decir 
        Sección II B del BOE
        Uso
        self.obtener_lista_urls_pdf()
        """
        lista_urls_pdf = []
        for XML in self.lista_xmls:
            textos = ""
            soup = BeautifulSoup(XML, "xml") 
            url_pdf = soup.find_all("url_pdf")           
            lista_urls_pdf.append(f'{self.dominio}{str(self.quitar_etiquetas_html(str(url_pdf)))}')
        self.lista_urls_pdf = lista_urls_pdf


    def generar_dataset(self) -> int:
        """
        Con los parámetros obtenidos de establecer_offset, generamos el dataset pandas
        de las disposiciones relativas a las ofertas de empelo público es decir 
        Sección II B del BOE
        Uso
        self.generar_dataset()
        Salida: Conteo de filas del dataset
        """
        self.buscar_urls_xmls()
        self.obtener_lista_xmls()
        self.obtener_lista_titulos()
        self.obtener_lista_textos()
        self.obtener_lista_urls_pdf()
        dataset_capturado = pd.DataFrame({'url':self.lista_urls_pdf, 
                                          'titulo':self.lista_titulos,
                                          'texto':self.lista_textos})
        
        self.dataset_boes = pd.concat([self.dataset_boes, dataset_capturado], ignore_index=True)
        return self.dataset_boes.shape[0]


    def obtener_dataset_final(self):
        """
        Finalmente devolvemos a la rutina principal el contenido del dataset completo
        MiClase.obtener_dataset_final()
        Salida: Dataset Completo
        """        
        return self.dataset_boes          
        

Obtenemos al menos 100 ofertas de emepleo publicadas en el BOE para facilitar busqueda de ofertas via API REST con Flask

In [4]:
# Rutina principal
N_REGISTROS_MINIMO = 100 # Limitamos a 100 las ofertas de empleo ya que si no se demora mucho
if __name__ == "__main__":    
    BOE = DescargaBOE()
    i = 0
    while True:      
        BOE.establecer_offset(i)        
        if(BOE.generar_dataset() > N_REGISTROS_MINIMO):
            break
        i += 1
    BOE.obtener_dataset_final()

# Obtenemos el dataset
df = BOE.obtener_dataset_final()

In [5]:
# df.to_csv('datos_offline/datos_obtenidos.boe.csv', sep='|', index=None)

In [ ]:
# Iniciamos API Rest Flask
# Crea una instancia de la aplicación Flask
app = Flask(__name__)

# Ruta para obtener la info de la peticion
@app.route('/opciones', methods=['GET'])
def obtener_elementos():
    """
    Método de test que devuelve la cabecera de la petición en formato JSON y el User-Aget,
    con esto se podría por ejemplo limitar las peticiones webscrapping a determinados navegadores.
    """
    return jsonify({'Headers': str(request.headers),
                    'User-agent': str(request.headers['User-Agent']),
                    'Opciones': '1'})

@app.route('/buscar', methods=['GET'])
def buscar_oferta_empleo():
    """
    Método que es ejecutado cuando se llama al API de la siguiente forma:
    http://URL/buscar?busqueda=<Termino de Busqueda>
    Busca y obtiene los BOES donde aparece dicho termino
    """
    termino_busqueda=request.args.get('busqueda')
    if termino_busqueda is None:
        return jsonify({'error': 'Se requieren parametro buscar'}), 400
    
    # Procesar los parámetros (aquí puedes realizar cualquier lógica que desees)
    df_resultado = df[df['texto'].str.contains(termino_busqueda)]
    
    # Devolver una respuesta JSON con el resultado
    return jsonify({'url': [url for url in df_resultado['url']],
                   'cuerpo': [texto for texto in df_resultado['texto']],
                   'titulo': [titulo for titulo in df['titulo']]}), 200

@app.route('/index')
def estado_models():
    return render_template('index.html')

@app.route('/docs/<path:filename>')
def imagen(filename):
    directorio_docs = os.path.join(app.root_path, 'static', 'docs')
    return send_from_directory(directorio_docs, filename)

if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Nov/2024 08:26:42] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [15/Nov/2024 08:26:42] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Nov/2024 08:26:53] "GET /static HTTP/1.1" 404 -
127.0.0.1 - - [15/Nov/2024 08:26:57] "GET /docs HTTP/1.1" 404 -
127.0.0.1 - - [15/Nov/2024 08:27:06] "GET /static/docs HTTP/1.1" 404 -
127.0.0.1 - - [15/Nov/2024 08:27:18] "GET /index HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2024 08:27:21] "GET /docs/PRACTICA-ETL_Descarga_Ofertas_de_empleo_API_Flask_y_Cliente.pdf HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2024 08:27:53] "GET /docs/Diagrama.png HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2024 08:29:02] "GET /docs/PRACTICA-ETL_Descarga_Ofertas_de_empleo_API_Flask_y_Cliente.pdf HTTP/1.1" 304 -
127.0.0.1 - - [15/Nov/2024 08:29:51] "GET /buscar?busqueda=Albacete HTTP/1.1" 200 -
